In [ ]:
import matplotlib.pyplot as plt
from quantopian.research.experimental import continuous_future, history
import pandas as pd
import numpy as np


In [ ]:
def get_fut_hist(symbol, start, end):
    df = history(
        [continuous_future(symbol, offset=i,  adjustment=None) for i in range(5)],
        fields=['price', 'contract'],
        frequency='daily', 
        start=start, 
        end=end
    )

    df.columns = ['f{}'.format(i) for i in range(1, 6)]
    return df



In [ ]:
ho = get_fut_hist('CL', '2011-01-01', '2018-6-15')


In [ ]:
contract = ho.contract
price = ho.price
contract.columns = contract.columns.map(lambda x: "f{}".format(x.offset))
price.columns = price.columns.map(lambda x: "f{}".format(x.offset))


In [ ]:
expdate = contract.applymap(lambda x:x.expiration_date)
expdate['today'] = expdate.index
remain_date =pd.concat([(expdate[c] - expdate['today']).dt.days for c in expdate.columns], axis=1)


In [ ]:
curve = pd.concat([price[c] / price[price.columns[0]] for c in price.columns], axis=1)
curve.columns = price.columns
curve

In [ ]:
remain_date_ratio = remain_date['f0'] / (remain_date['f1'] - remain_date['f0'] )


In [ ]:
price['interporate f0'] = (price['f0'] * remain_date_ratio + price['f1'] * (1-remain_date_ratio))
price['interporate f1'] = (price['f1'] * remain_date_ratio + price['f2'] * (1-remain_date_ratio))
price['interporate f2'] = (price['f2'] * remain_date_ratio + price['f3'] * (1-remain_date_ratio))
price['interporate f3'] = (price['f3'] * remain_date_ratio + price['f4'] * (1-remain_date_ratio))


In [ ]:
(price['f1'] - price['interporate f1']).plot()
(price['f2'] - price['interporate f2']).plot()
(price['f3'] - price['interporate f3']).plot()
price['f3'].plot(secondary_y=True)
# price['interporate f3'].plot(secondary_y=True)

